In [39]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import os
import pandas as pd


In [85]:
def setup():
    driver = webdriver.Chrome()
    page_url="https://community.shopify.com/c/shopify-apps/bd-p/shopify-apps"
    driver.get(page_url)
    return driver

def teardown(driver):
    driver.quit()

def save_data_to_file(data, filename):
    if not os.path.exists('data'):
        os.makedirs('data')
    
    with open(f'data/{filename}', 'w', encoding='utf-8') as file:
        file.write(data)


url=[]

def scraper():    
    driver = setup()
    driver.implicitly_wait(0.5)
    count=1
    
    for page_no in range(1,7):
        if(page_no==1):
            data=driver.find_element(By.CLASS_NAME,"main-body").find_elements(By.CSS_SELECTOR,"div.res-data")
        else:
            next_page_container=driver.find_element(By.CLASS_NAME,"pagination-div")
            
            next_page=WebDriverWait(next_page_container, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,".pagination-div:last-child")))
            print(next_page.get_attribute("outerHTML"))
            next_page.click()            
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,".main-body")))
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.res-data")))
            data=driver.find_elements(By.CSS_SELECTOR,"div.res-data")
            
            
        for i,eles in enumerate(data):
            try:
                data = driver.find_elements(By.CSS_SELECTOR, "div.res-data")
                divs=data[i]
                link=divs.find_element(By.CSS_SELECTOR,"div.res-subject > a")
                post_url=link.get_attribute("href")
                url.append(post_url)
                
                link.click()
                element=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.lia-quilt-row-main")))
                data=element.get_attribute("outerHTML")
                save_data_to_file(data, f'page_{count}.html')
                count=count+1
                # driver.execute_script("window.history.go(-1)")
                driver.back()
                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "res-data")))
                
            except NoSuchElementException as e:
                    print(f"Element not found in div {i+1}: {e}")
            except TimeoutException as e:
                    print(f"Timeout occurred for div {i+1}: {e}")
            except Exception as e:
                    print("error at",i+1)
                    count=count+1

                
    teardown(driver)
    
scraper()

error at 16
error at 18
error at 20


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6766FB5D2+29090]
	(No symbol) [0x00007FF67666E689]
	(No symbol) [0x00007FF67652B1CA]
	(No symbol) [0x00007FF67657EFD7]
	(No symbol) [0x00007FF67657F22C]
	(No symbol) [0x00007FF6765727CC]
	(No symbol) [0x00007FF6765A672F]
	(No symbol) [0x00007FF6765726A6]
	(No symbol) [0x00007FF6765A6900]
	(No symbol) [0x00007FF6765C65D9]
	(No symbol) [0x00007FF6765A6493]
	(No symbol) [0x00007FF6765709B1]
	(No symbol) [0x00007FF676571B11]
	GetHandleVerifier [0x00007FF676A18C5D+3295277]
	GetHandleVerifier [0x00007FF676A64843+3605523]
	GetHandleVerifier [0x00007FF676A5A707+3564247]
	GetHandleVerifier [0x00007FF6767B6EB6+797318]
	(No symbol) [0x00007FF67667980F]
	(No symbol) [0x00007FF6766753F4]
	(No symbol) [0x00007FF676675580]
	(No symbol) [0x00007FF676664A1F]
	BaseThreadInitThunk [0x00007FFEF598257D+29]
	RtlUserThreadStart [0x00007FFEF6C4AF28+40]


In [72]:
d={"title":[],"user_name":[],"user_title":[],"url":url,"user_profile_link":[],"number_of_posts_by_user":[],"number_of_solutions_by_user":[],"number_of_kudos_by_user":[],"timestamp":[],"post_content":[],"solved":[],"labels_list":[],"number_of_views":[],"number_of_likes":[],"replies":[]}

for file in os.listdir("data"):
    with open(f"data/{file}",encoding="utf8") as f:
        html_doc=f.read()
    soup=BeautifulSoup(html_doc,"html.parser")    
    
    title=soup.find("h1",class_="page-title")
    d["title"].append(title.text)
    
    user_name=soup.find("a",class_="lia-user-name-link")
    d["user_name"].append(user_name.text)
    
    user_profile_link=user_name.get_attribute_list("href")
    d["user_profile_link"].append(user_profile_link[0])
   
    user_title_container=soup.find("span",class_="lia-message-unread")
    user_title=user_title_container.find_next("div")    
    d["user_title"].append(user_title.text.strip())
    
    number_of_posts_by_user=soup.find(class_="lia-message-author-posts-count")
    d["number_of_posts_by_user"].append(number_of_posts_by_user.text)
    
    number_of_solutions_by_user=soup.find(class_="lia-message-author-solutions-count")
    d["number_of_solutions_by_user"].append(number_of_solutions_by_user.text.strip())
    
    nunber_of_kudos_by_user=soup.find(class_="lia-message-author-kudos-count")
    d["number_of_kudos_by_user"].append(nunber_of_kudos_by_user.text.strip())
    
    publish_date=soup.select(".DateTime > meta")[0].get_attribute_list("content")
    d["timestamp"].append(publish_date[0])
    
    post_content_data=soup.find(class_="lia-message-body-content")
    for content in post_content_data(['style', 'script']):
        content.decompose().strip()
    post_content=' '.join(post_content_data.stripped_strings)
    d["post_content"].append(post_content)    
    
    if soup.find(class_="lia-panel-feedback-banner-safe"):
       solve="Y"
    else:
       solve="N"    
    d["solved"].append(solve)
    
    label_data=soup.find_all(class_="label")
    labels=[]
    for label_arr in label_data:
        labels.append((label_arr.text).strip())
    d["labels_list"].append(labels)
        
    view_count=soup.find(class_="lia-message-stats-count")
    d["number_of_views"].append(view_count.text.strip())    
    
    like_count=soup.find(class_="MessageKudosCount")
    d["number_of_likes"].append(like_count.text.strip())  
    
    replies=soup.find(class_="threaded-detail-message-list")
    replies=soup.find(class_="threaded-detail-message-list")
    replies=soup.find(class_="lia-threaded-detail-display-message-view")
    reply_list=[]
    if replies:
        reply_data=soup.find_all(class_="lia-message-body-content")
        for content in reply_data[1](['style', 'script']):
            content.decompose().strip()
        reply_content=' '.join(reply_data[1].stripped_strings)
        reply_list.append(reply_content)
    else:
        pass
    d["replies"].append(reply_list)

  
    
df=pd.DataFrame(data=d);
df.to_csv("data.csv")    

ValueError: All arrays must be of the same length